## Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost


In [2]:
# load data
train_csv_file = 'DataFiles/CreditCard_train.csv'
test_csv_file = 'DataFiles/CreditCard_test.csv'

_train_data = pd.read_csv(train_csv_file, index_col=0, header=1)
_test_data = pd.read_csv(test_csv_file, index_col=0, header=1)

# create copy df 
df_train = _train_data.copy()
df_test = _test_data.copy()

df_train.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,24000.000000,24000.000000,24000.000000,24000.00000,24000.000000,24000.000000,24000.00000,24000.000000,24000.000000,24000.000000,...,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.00000,24000.000000,24000.000000,24000.000000,24000.000000
mean,165495.986667,1.628250,1.847417,1.55725,35.380458,-0.003125,-0.12350,-0.154750,-0.211667,-0.252917,...,42368.188417,40000.682542,38563.710625,5542.912917,5.815336e+03,4969.26600,4743.480042,4783.486042,5189.399042,0.223750
std,129128.744855,0.483282,0.780007,0.52208,9.271050,1.123425,1.20058,1.204033,1.166549,1.136993,...,63070.680934,60345.012766,59155.759799,15068.576072,2.079703e+04,16095.61434,14883.269990,15270.405279,17630.371990,0.416765
min,10000.000000,1.000000,0.000000,0.00000,21.000000,-2.000000,-2.00000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.00000,28.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,...,2340.000000,1740.000000,1234.750000,1000.000000,8.000000e+02,379.00000,279.750000,244.000000,60.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.00000,34.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,18940.500000,18107.500000,17036.000000,2100.000000,2.000000e+03,1702.50000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.00000,41.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,52188.500000,49746.500000,48796.250000,5000.000000,5.000000e+03,4347.25000,4000.000000,4005.000000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.00000,79.000000,8.000000,8.00000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,505000.000000,1.684259e+06,896040.00000,497000.000000,417990.000000,528666.000000,1.000000


### feature selection

In [3]:
from sklearn.preprocessing import StandardScaler

X_train_unscaled = df_train.iloc[:,:-1]
y_train_unraveled = df_train.iloc[:,-1:]

X_test_unscaled = df_test.iloc[:,:-1]
y_test_unraveled = df_test.iloc[:,-1:]


# standardise data
X_train = X_train_unscaled
X_test = X_test_unscaled

# standardise test data
y_train = y_train_unraveled.values.ravel()
y_test = y_test_unraveled.values.ravel()

In [4]:
y_train
X_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,689,0,0,0,0,689,0,0,0,0
2,120000,2,2,2,26,-1,2,0,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
3,90000,2,2,2,34,0,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
4,50000,2,2,1,37,0,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
5,50000,1,2,1,57,-1,0,-1,0,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23996,80000,1,2,1,25,1,2,2,0,0,...,80903,80215,63296,49854,3800,6,3636,2646,2000,1830
23997,20000,1,2,1,25,0,0,0,0,0,...,17562,17322,17119,17350,1552,2659,1419,606,500,1000
23998,10000,1,2,2,26,0,0,0,0,0,...,9825,17506,16608,9176,1300,2200,1300,320,1820,1000


In [5]:
type(X_train)

pandas.core.frame.DataFrame

In [6]:
X_train.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
count,24000.000000,24000.000000,24000.000000,24000.00000,24000.000000,24000.000000,24000.00000,24000.000000,24000.000000,24000.000000,...,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.00000,24000.000000,24000.000000,24000.000000
mean,165495.986667,1.628250,1.847417,1.55725,35.380458,-0.003125,-0.12350,-0.154750,-0.211667,-0.252917,...,4.636706e+04,42368.188417,40000.682542,38563.710625,5542.912917,5.815336e+03,4969.26600,4743.480042,4783.486042,5189.399042
std,129128.744855,0.483282,0.780007,0.52208,9.271050,1.123425,1.20058,1.204033,1.166549,1.136993,...,6.819390e+04,63070.680934,60345.012766,59155.759799,15068.576072,2.079703e+04,16095.61434,14883.269990,15270.405279,17630.371990
min,10000.000000,1.000000,0.000000,0.00000,21.000000,-2.000000,-2.00000,-2.000000,-2.000000,-2.000000,...,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.00000,28.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,...,2.773500e+03,2340.000000,1740.000000,1234.750000,1000.000000,8.000000e+02,379.00000,279.750000,244.000000,60.750000
50%,140000.000000,2.000000,2.000000,2.00000,34.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,2.003900e+04,18940.500000,18107.500000,17036.000000,2100.000000,2.000000e+03,1702.50000,1500.000000,1500.000000,1500.000000
75%,240000.000000,2.000000,2.000000,2.00000,41.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,5.929800e+04,52188.500000,49746.500000,48796.250000,5000.000000,5.000000e+03,4347.25000,4000.000000,4005.000000,4000.000000
max,1000000.000000,2.000000,6.000000,3.00000,79.000000,8.000000,8.00000,8.000000,8.000000,8.000000,...,1.664089e+06,891586.000000,927171.000000,961664.000000,505000.000000,1.684259e+06,896040.00000,497000.000000,417990.000000,528666.000000


### scaling data

In [8]:
from sklearn import preprocessing 
 
X_train = preprocessing.scale(X_train) #removes columns and transforms data into ndarray
X_test = preprocessing.scale(X_test)

In [12]:
X_train

array([[-1.12677482,  0.76923536,  0.19562207, ..., -0.31871886,
        -0.31325859, -0.29435041],
       [-0.35233777,  0.76923536,  0.19562207, ..., -0.25152793,
        -0.31325859, -0.18090744],
       [-0.58466889,  0.76923536,  0.19562207, ..., -0.25152793,
        -0.24777108, -0.01074299],
       ...,
       [-1.20421852, -1.29999224,  0.19562207, ..., -0.29721776,
        -0.19407132, -0.23762892],
       [-1.12677482, -1.29999224,  4.04182335, ..., -0.25152793,
        -0.31325859, -0.29435041],
       [-0.50722518, -1.29999224, -1.08644502, ..., -0.28512339,
        -0.24777108, -0.23762892]])

### Gradient Boosting

In [15]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X_train,y_train)

y_train2 = y_train-tree_reg1.predict(X_train)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train,y_train2)

y_train3 = y_train2-tree_reg2.predict(X_train)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X_train,y_train3)

y_pred = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))
print(accuracy_score(y_test, y_pred.round()))



0.8296666666666667


In [28]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=50, learning_rate=1)
gbrt.fit(X_train, y_train)
y_pred = gbrt.predict(X_test)
print("Train accuracy score: ", accuracy_score(y_train,gbrt.predict(X_train).round()))
print("Test accuracy score: ", accuracy_score(y_test,y_pred.round()))

Train accuracy score:  0.825125
Test accuracy score:  0.8283333333333334


In [ ]:
y_pred

### xgboost

In [45]:
import xgboost


xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_test)
print(accuracy_score(y_test,y_pred.round()))

y_train_pred = 



0.8211666666666667


In [42]:
X_train_unscaled.describe()

X_test_unscaled.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.00000,6000.000000,6000.00000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6.000000e+03,6000.000000,6000.000000,6000.000000,6000.000000
mean,175437.666667,1.505667,1.876000,1.530333,35.905667,-0.07100,-0.174833,-0.21200,-0.256667,-0.319333,...,49597.543500,46841.991167,41554.274667,40103.959500,6146.250833,6.344473e+03,6251.343500,5156.464167,4862.994000,5319.916667
std,131905.841315,0.500010,0.830104,0.521017,8.990675,1.12376,1.182723,1.16674,1.178850,1.116348,...,73739.999695,69041.251707,62562.774782,61111.043232,21523.044734,3.040158e+04,22638.732479,18466.518562,15310.976004,18355.191015
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.00000,-2.000000,-2.00000,-2.000000,-2.000000,...,-46127.000000,-65167.000000,-61372.000000,-209051.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,60000.000000,1.000000,1.000000,1.000000,29.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.000000,...,2487.750000,2290.750000,1900.000000,1320.000000,1000.000000,9.900000e+02,799.750000,326.000000,278.750000,288.750000
50%,150000.000000,2.000000,2.000000,2.000000,34.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,20360.500000,19657.000000,18071.500000,17203.500000,2195.500000,2.230000e+03,2000.000000,1634.500000,1590.000000,1697.000000
75%,250000.000000,2.000000,2.000000,2.000000,42.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,65252.750000,61357.750000,52099.000000,50620.500000,5188.750000,5.042250e+03,5000.000000,4209.500000,4145.000000,4300.000000
max,780000.000000,2.000000,6.000000,3.000000,75.000000,8.00000,7.000000,8.00000,7.000000,7.000000,...,855086.000000,706864.000000,587067.000000,514975.000000,873552.000000,1.227082e+06,889043.000000,621000.000000,426529.000000,443001.000000


In [46]:
X_train_unscaled.describe()


xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train_unscaled_data,y_train)
y_pred = xgb_reg.predict(X_test_unscaled_data)
print(accuracy_score(y_test,y_pred.round()))

0.8261666666666667


In [30]:
y_train

array([1, 1, 0, ..., 0, 0, 1])

In [31]:
X_train

array([[-1.12677482,  0.76923536,  0.19562207, ..., -0.31871886,
        -0.31325859, -0.29435041],
       [-0.35233777,  0.76923536,  0.19562207, ..., -0.25152793,
        -0.31325859, -0.18090744],
       [-0.58466889,  0.76923536,  0.19562207, ..., -0.25152793,
        -0.24777108, -0.01074299],
       ...,
       [-1.20421852, -1.29999224,  0.19562207, ..., -0.29721776,
        -0.19407132, -0.23762892],
       [-1.12677482, -1.29999224,  4.04182335, ..., -0.25152793,
        -0.31325859, -0.29435041],
       [-0.50722518, -1.29999224, -1.08644502, ..., -0.28512339,
        -0.24777108, -0.23762892]])

#### kernel PCA

In [7]:
from sklearn.decomposition import KernelPCA

#transformer = KernelPCA(kernel='poly', degree=3)
#X_train_transformed = transformer.fit_transform(X_train[0:4000])
#X_train_transformed.shape

## Testing some classifiers

In [10]:
log_clf = LogisticRegression(max_iter=10000)
rnd_clf = RandomForestClassifier()
svm_clf = SVC(gamma='auto',C=4)
#svm_kernel_clf = SVC(kernel='poly', degree=2)
#from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import make_pipeline
#svm_scaled_clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

for clf in (log_clf, rnd_clf, svm_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test,y_pred))

LogisticRegression 0.821
RandomForestClassifier 0.828
SVC 0.8315


In [ ]:
X_train.describe()

## HORROR

 ## Horror END


end

### Voting classifier

In [11]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svm', svm_clf)],
    voting='hard')
voting_clf.fit(X_train,y_train)
y_pred = voting_clf.predict(X_test)
print(voting_clf.__class__.__name__, accuracy_score(y_test,y_pred))

VotingClassifier 0.829


### Adaboosting

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8303333333333334


In [14]:
y_train_pred = ada_clf.predict(X_train)
print('training score accuracy = ' + str(accuracy_score(y_train, y_train_pred)))

training score accuracy = 0.8171666666666667


### NN - incomplete